In [1]:
import bw2calc as bc
import bw2data as bd
import bw2regional as bwr
import bw2analyzer as ba
import csv
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
bd.projects.set_current("LC IMPACT case study")

In [3]:
for act in bd.Database("Fuel comparison"):
    print(act.key, act)

('Fuel comparison', 'maize transport') 'Driving with maize' (kilometer, ('ecoinvent', 'RER'), None)
('Fuel comparison', 'petrol transport') 'Driving with petrol' (kilometer, ('ecoinvent', 'RER'), None)
('Fuel comparison', 'sugarcane transport') 'Driving with sugarcane' (kilometer, ('ecoinvent', 'RER'), None)
('Fuel comparison', 'sugarcane') 'Sugarcane ethanol in europe' (kilogram, ('ecoinvent', 'RER'), None)
('Fuel comparison', 'ethanol driving without fuel') 'ethanol driving without fuel' (kilometer, ('ecoinvent', 'RER'), None)
('Fuel comparison', 'maize') 'Maize ethanol in europe' (kilogram, ('ecoinvent', 'RER'), None)


In [4]:
[x for x in bd.methods if x[0] == 'LC-IMPACT']

[('LC-IMPACT', 'Land Use', 'Occupation', 'Marginal', 'Certain'),
 ('LC-IMPACT', 'Land Use', 'Occupation', 'Average', 'Certain'),
 ('LC-IMPACT', 'Land Use', 'Transformation', 'Marginal', 'Certain'),
 ('LC-IMPACT', 'Land Use', 'Transformation', 'Marginal', 'All'),
 ('LC-IMPACT', 'Land Use', 'Transformation', 'Average', 'Certain'),
 ('LC-IMPACT', 'Land Use', 'Transformation', 'Average', 'All'),
 ('LC-IMPACT', 'Particulate Matter Formation', 'Marginal', 'All'),
 ('LC-IMPACT', 'Particulate Matter Formation', 'Marginal', 'Certain'),
 ('LC-IMPACT', 'Water Use', 'Human Health', 'Average'),
 ('LC-IMPACT', 'Water Use', 'Human Health', 'Marginal'),
 ('LC-IMPACT',
  'Water Use',
  'Ecosystem Quality',
  'Surface Water',
  'Marginal',
  'Certain'),
 ('LC-IMPACT',
  'Water Use',
  'Ecosystem Quality',
  'Surface Water',
  'Marginal',
  'All')]

In [5]:
irrigation = [x.id for x in bd.Database("ecoinvent") if 'irrigation' in x['name']]
crops = [x.id for x in bd.Database("ecoinvent") 
         if x['name'] in ('sugarcane production', 'maize grain production')]

In [6]:
activities = [
    ('Fuel comparison', 'sugarcane transport'),
    ('Fuel comparison', 'maize transport'),
    ('Fuel comparison', 'petrol transport'),
]

lcimpact = {
 # ("LC-IMPACT", "Climate Change", "Human Health", "Marginal", "Certain", "100 Years"): None,
 # ("LC-IMPACT", "Climate Change", "Human Health", "Marginal", "All", "100 Years"): None,
 # ("LC-IMPACT", "Climate Change", "Human Health", "Marginal", "Certain", "Infinite"): None,
 # ("LC-IMPACT", "Climate Change", "Human Health", "Marginal", "All", "Infinite"): None,
 # ("LC-IMPACT", "Climate Change", "Terrestrial Ecosystems", "Marginal", "Certain", "100 Years"): None,
 # ("LC-IMPACT", "Climate Change", "Terrestrial Ecosystems",  "Marginal", "All", "100 Years"): None,
 # ("LC-IMPACT", "Climate Change", "Terrestrial Ecosystems",  "Marginal", "Certain", "Infinite"): None,
 # ("LC-IMPACT", "Climate Change", "Terrestrial Ecosystems",  "Marginal", "All", "Infinite"): None,
 # ("LC-IMPACT", "Climate Change", "Aquatic Ecosystems", "Marginal", "All", "100 Years"): None,
 # ("LC-IMPACT", "Climate Change", "Aquatic Ecosystems",  "Marginal", "All", "Infinite"): None, 
 ('LC-IMPACT', 'Land Use', 'Occupation', 'Marginal', 'Certain'): {
     'xt others': 'world-topo-ecoregions-weighted-pop-density',
     'activities': crops,
     'sugarcane': 'world-topo-ecoregions-sugarcane_landuse_intensity',
     'maize': 'world-topo-ecoregions-maize_landuse_intensity',
 },
 ('LC-IMPACT', 'Particulate Matter Formation', 'Marginal', 'All'): {
     'xt others': 'world-topo-particulate-matter-weighted-pop-density',
     'activities': [],
     'sugarcane': 'world-topo-particulate-matter-sugarcane_landuse_intensity',
     'maize': 'world-topo-particulate-matter-maize_landuse_intensity',
 },
 ('LC-IMPACT', 'Particulate Matter Formation', 'Marginal', 'Certain'): {
     'xt others': 'world-topo-particulate-matter-weighted-pop-density',
     'activities': [],
     'sugarcane': 'world-topo-particulate-matter-sugarcane_landuse_intensity',
     'maize': 'world-topo-particulate-matter-maize_landuse_intensity',
 },
 ('LC-IMPACT', 'Water Use', 'Ecosystem Quality', 'Surface Water', 'Marginal', 'All'): {
     'xt others': 'world-topo-watersheds-eq-sw-all-weighted-pop-density',
     'activities': irrigation + crops,
     'sugarcane': 'world-topo-watersheds-eq-sw-all-sugarcane_water_intensity',
     'maize': 'world-topo-watersheds-eq-sw-all-maize_water_intensity',
 },
 ('LC-IMPACT', 'Water Use', 'Ecosystem Quality', 'Surface Water', 'Marginal', 'Certain'): {
     'xt others': 'world-topo-watersheds-eq-sw-certain-weighted-pop-density',
     'activities': irrigation + crops,
     'sugarcane': 'world-topo-watersheds-eq-sw-certain-sugarcane_water_intensity',
     'maize': 'world-topo-watersheds-eq-sw-certain-maize_water_intensity',
 },
 ('LC-IMPACT', 'Water Use', 'Human Health', 'Marginal'): {
     'xt others': 'world-topo-watersheds-hh-weighted-pop-density',
     'activities': irrigation + crops,
     'sugarcane': 'world-topo-watersheds-hh-sugarcane_water_intensity',
     'maize': 'world-topo-watersheds-hh-maize_water_intensity',
 },
}

In [9]:
method = ('LC-IMPACT', 'Land Use', 'Occupation', 'Marginal', 'Certain')

xt_ag = bwr.ExtensionTablesLCA(
    {('Fuel comparison', 'sugarcane transport'): 1},
    method,
    xtable=lcimpact[method]['sugarcane'],
    limitations={
        'activities': lcimpact[method]['activities'],
    }
)
xt_ag.lci()
xt_ag.lcia()
xt_ag.score

2.7066301232545834e-14

In [8]:
xt_ag.geodataframe_xtable_spatial_scale().shape

(1308, 8)

In [8]:
xt_ag.geodataframe_xtable_spatial_scale(cutoff=0.001).shape

(20, 8)

In [11]:
# xt_ag.geodataframe_xtable_spatial_scale().explore(column='score_rel')

In [12]:
bd.databases

Databases dictionary with 3 object(s):
	Fuel comparison
	biosphere3
	ecoinvent

In [13]:
[x for x in bd.Database("ecoinvent") if x['location'] == "RU"]

['electricity production, nuclear, pressure water reactor' (kilowatt hour, RU, None),
 'market for hard coal' (kilogram, RU, None),
 'treatment of blast furnace gas, in power plant' (kilowatt hour, RU, None),
 'sunflower production' (kilogram, RU, None),
 'electricity, high voltage, import from GE' (kilowatt hour, RU, None),
 'uranium production, centrifuge, enriched 4.0%' (unit, RU, None),
 'platinum group metal mine operation, ore with high palladium content' (kilogram, RU, None),
 'market for nitric acid, without water, in 50% solution state' (kilogram, RU, None),
 'natural gas production' (cubic meter, RU, None),
 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014' (megajoule, RU, None),
 'natural gas, burned in gas turbine, for compressor station' (kilowatt hour, RU, None),
 'electricity production, photovoltaic, 3kWp slanted-roof installation, single-Si, panel, mounted' (kilowatt hour, RU, None),
 'market for inorganic phosphorus fertiliser, as P2O5' (kilog

In [8]:
xt_ag.geodataframe_xtable_spatial_scale().to_file('xt_scale.gpkg')

In [9]:
xt_ag.geodataframe_xtable_spatial_scale(sum_flows=False).to_file('xt_scale_unsummed.gpkg')

In [10]:
xt_ag.geodataframe_xtable_spatial_scale(annotate_flows=True).to_file('xt_scale_annotated.gpkg')

In [ ]:
xt_ag.geodataframe_ia_spatial_scale().to_file('ia_scale.gpkg')

In [8]:
xt_ag.geodataframe_inv_spatial_scale().to_file('inv_scale.gpkg')

In [ ]:
results = []

In [ ]:
HEADERS = ["Name", "Unit", "Ref. product", "Location", "Score (absolute)", "Score (fraction of total)"]

In [ ]:
for method, meta in lcimpact.items():
    print(method)
    results.append(list(method))
    
    fu = ('Fuel comparison', 'petrol transport')
    lca = bc.LCA({fu: 1}, method)
    lca.lci()
    lca.lcia()
    
    results.append([])
    results.append(list(fu))
    results.append(["Site-generic score", lca.score])
    results.append(["Top 10 contributing activities"])
    results.append(HEADERS)
    for score, _, act in bwa.ContributionAnalysis().annotated_top_processes(lca)[:10]:
        results.append([act['name'], act['unit'], act['reference product'], act['location'], score, score / lca.score])

    if method[1] != 'Climate Change':        
        xt = bwr.ExtensionTablesLCA(
            {fu: 1},
            method,
            xtable=meta['xt others']
        )
        xt.lci()
        xt.lcia()

        results.append([])
        results.append(["Site-dependent score", xt.score])
        results.append(["Top 10 contributing activities"])
        results.append(HEADERS)
        for score, _, act in ContributionAnalysis().annotated_top_processes(xt)[:10]:
            results.append([act['name'], act['unit'], act['reference product'], act['location'], score, score / xt.score])

    for fu, crop in (
            (('Fuel comparison', 'sugarcane transport'), 'sugarcane'),
            (('Fuel comparison', 'maize transport'), 'maize'),
            ):
        
        lca = LCA({fu: 1}, method)
        lca.lci()
        lca.lcia()

        results.append([])
        results.append(list(fu))
        results.append(["Site-generic score", lca.score])
        results.append(["Top 10 contributing activities"])
        results.append(HEADERS)
        for score, _, act in ContributionAnalysis().annotated_top_processes(lca)[:10]:
            results.append([act['name'], act['unit'], act['reference product'], act['location'], score, score / lca.score])

        if method[1] != 'Climate Change':        
            if meta['activities']:
                xt_ag = ExtensionTablesLCA(
                    {fu: 1},
                    method,
                    xtable=meta[crop],
                    limitations={
                        'activities': meta['activities'],
                    }
                )
                xt_ag.lci()
                xt_ag.lcia()

                if method == ('LC-IMPACT', 'Water Use', 'Ecosystem Quality', 'Surface Water', 'Marginal', 'All'):
                    as_ia_spatial_scale(xt_ag, "watersheds-eq-sw-all", crop + "_water_ag.geojson")
                elif method == ('LC-IMPACT', 'Land Use', 'Occupation', 'Marginal', 'Certain'):
                    as_ia_spatial_scale(xt_ag, "ecoregions", crop + "_land_ag.geojson")
                    
                xt_others = ExtensionTablesLCA(
                    {fu: 1},
                    method,
                    xtable=meta['xt others'],
                    limitations={
                        'activities': meta['activities'],
                        'activities mode': 'exclude'
                    }
                )
                xt_others.lci()
                xt_others.lcia()

                if method == ('LC-IMPACT', 'Water Use', 'Ecosystem Quality', 'Surface Water', 'Marginal', 'All'):
                    as_ia_spatial_scale(xt_others, "watersheds-eq-sw-all", crop + "_water_others.geojson")
                elif method == ('LC-IMPACT', 'Land Use', 'Occupation', 'Marginal', 'Certain'):
                    as_ia_spatial_scale(xt_others, "ecoregions", crop + "_land_others.geojson")

                xt_ag.characterized_inventory += xt_others.characterized_inventory

                results.append([])
                results.append(["Site-dependent score", xt_ag.score])
                results.append(["Top 10 contributing activities"])
                results.append(HEADERS)

                for score, _, act in ContributionAnalysis().annotated_top_processes(xt_ag)[:10]:
                    results.append([act['name'], act['unit'], act['reference product'], act['location'], score, score / lca.score])

                if method == ('LC-IMPACT', 'Water Use', 'Ecosystem Quality', 'Surface Water', 'Marginal', 'All'):
                    add_two_geojson_results(
                        crop + "_water_ag.geojson", 
                        crop + "_water_others.geojson", 
                        crop + "_water_total.geojson",
                    )
                elif method == ('LC-IMPACT', 'Land Use', 'Occupation', 'Marginal', 'Certain'):
                    add_two_geojson_results(
                        crop + "_land_ag.geojson", 
                        crop + "_land_others.geojson", 
                        crop + "_land_total.geojson",
                    )
                    
            else:
                xt = ExtensionTablesLCA(
                    {fu: 1},
                    method,
                    xtable=meta['xt others']
                )
                xt.lci()
                xt.lcia()

                results.append([])
                results.append(["Site-dependent score", xt.score])
                results.append(["Top 10 contributing activities"])
                results.append(HEADERS)
                for score, _, act in ContributionAnalysis().annotated_top_processes(xt)[:10]:
                    results.append([act['name'], act['unit'], act['reference product'], act['location'], score, score / xt.score])

    results.append([])
    results.append([])

In [ ]:
with open("lcimpact-case-study.csv", "w") as f:
    writer = csv.writer(f)
    for row in results:
        writer.writerow(row)